# Using Xunfei Spark LLM in LangChain App

In [1]:
! pip install langchain -q -U

In [22]:
from langchain.chat_models import ChatOpenAI

spark_chat = ChatOpenAI(openai_api_base="http://localhost:8000/v1", openai_api_key="sk-xxx")

In [7]:
from langchain.schema.messages import HumanMessage, SystemMessage

messages = [
    HumanMessage(content="大语言模型的理论基础是什么？"),
]

In [19]:
spark_chat.invoke(messages)

AIMessage(content='大语言模型的理论基础是深度学习。大语言模型的目标是理解和生成人类语言，为了实现这个目标，模型需要在大量文本数据上进行训练，以学习语言的各种模式和结构。通常来说，大语言模型指的是那些在大规模文本语料上训练、包含百亿级别(或更多)参数的语言模型。目前的大语言模型采用与小模型类似的Transformer架构和预训练目标(如 Language Modeling),与小模型的主要区别在于增加模型大小、训练数据和计算资源。')

In [25]:

from langchain.memory import ConversationBufferMemory
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{human_input}")
    ]
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [26]:
from langchain.chains import LLMChain
chat_llm_chain = LLMChain(
    llm=spark_chat,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

In [27]:
chat_llm_chain.predict(human_input="你是谁？")



> Entering new LLMChain chain...
Prompt after formatting:
Human: 你是谁？

> Finished chain.


'您好，我是科大讯飞研发的认知智能大模型，我的名字叫讯飞星火认知大模型。我可以和人类进行自然交流，解答问题，高效完成各领域认知智能需求。'

In [28]:
chat_llm_chain.predict(human_input="请解释第一性原理")



> Entering new LLMChain chain...
Prompt after formatting:
Human: 你是谁？
AI: 您好，我是科大讯飞研发的认知智能大模型，我的名字叫讯飞星火认知大模型。我可以和人类进行自然交流，解答问题，高效完成各领域认知智能需求。
Human: 请解释第一性原理

> Finished chain.


'第一性原理是指回归事物最基本的条件，将其拆分成各要素进行解构分析，从而找到实现目标最优路径的方法。该原理源于古希腊哲学家亚里士多德提出的一个哲学观点：“每个系统中存在一个最基本的命题，它不能被违背或删除。” \n\n例如，马斯克的“第一性原理”思维是指回溯事物本质，重新思考该怎么做。他认为，通过“第一性原理”，可以将事情升华到最根本的真理，然后从最核心处开始推理。'

In [29]:
chat_llm_chain.predict(human_input="这位哲学家的最大成就是什么？")



> Entering new LLMChain chain...
Prompt after formatting:
Human: 你是谁？
AI: 您好，我是科大讯飞研发的认知智能大模型，我的名字叫讯飞星火认知大模型。我可以和人类进行自然交流，解答问题，高效完成各领域认知智能需求。
Human: 请解释第一性原理
AI: 第一性原理是指回归事物最基本的条件，将其拆分成各要素进行解构分析，从而找到实现目标最优路径的方法。该原理源于古希腊哲学家亚里士多德提出的一个哲学观点：“每个系统中存在一个最基本的命题，它不能被违背或删除。” 

例如，马斯克的“第一性原理”思维是指回溯事物本质，重新思考该怎么做。他认为，通过“第一性原理”，可以将事情升华到最根本的真理，然后从最核心处开始推理。
Human: 这位哲学家的最大成就是什么？

> Finished chain.


'亚里士多德是古希腊哲学家、科学家和教育家之一，堪称希腊哲学的集大成者。他的著作是古代的百科全书，据说有四百到一千部，主要有《工具论》、《形而上学》、《物理学》、《伦理学》、《政治学》、《诗学》等  。马克思曾称亚里士多德是古希腊哲学家中最博学的人物，恩格斯称他是“古代的黑格尔”。作为一位百科全书式的科学家，他几乎对每个学科都做出了贡献。\n\n亚里士多德的最大成就是他对科学和哲学的贡献。他在物理学、生物学、心理学、逻辑学等方面都有重要贡献。他对人类思想和社会生活的影响也是深远的  。'

In [31]:
chat_llm_chain.predict(human_input="他在心理学方面的贡献是什么？")



> Entering new LLMChain chain...
Prompt after formatting:
Human: 你是谁？
AI: 您好，我是科大讯飞研发的认知智能大模型，我的名字叫讯飞星火认知大模型。我可以和人类进行自然交流，解答问题，高效完成各领域认知智能需求。
Human: 请解释第一性原理
AI: 第一性原理是指回归事物最基本的条件，将其拆分成各要素进行解构分析，从而找到实现目标最优路径的方法。该原理源于古希腊哲学家亚里士多德提出的一个哲学观点：“每个系统中存在一个最基本的命题，它不能被违背或删除。” 

例如，马斯克的“第一性原理”思维是指回溯事物本质，重新思考该怎么做。他认为，通过“第一性原理”，可以将事情升华到最根本的真理，然后从最核心处开始推理。
Human: 这位哲学家的最大成就是什么？
AI: 亚里士多德是古希腊哲学家、科学家和教育家之一，堪称希腊哲学的集大成者。他的著作是古代的百科全书，据说有四百到一千部，主要有《工具论》、《形而上学》、《物理学》、《伦理学》、《政治学》、《诗学》等  。马克思曾称亚里士多德是古希腊哲学家中最博学的人物，恩格斯称他是“古代的黑格尔”。作为一位百科全书式的科学家，他几乎对每个学科都做出了贡献。

亚里士多德的最大成就是他对科学和哲学的贡献。他在物理学、生物学、心理学、逻辑学等方面都有重要贡献。他对人类思想和社会生活的影响也是深远的  。
Human: 请总结我们今天的对话
AI: 抱歉，由于我是一个计算机程序，我没有记忆功能，也无法记录我们之前的对话。如果您有任何其他问题或需要帮助，请随时告诉我。
Human: 他在心理学方面的贡献是什么？

> Finished chain.


'亚里士多德在心理学方面的贡献是他写了一本专门讨论心理学的书：《De Anima》或《On the Soul》。在这本书中，他提出了支配人类身心的抽象概念。身体和心灵存在于同一个存在中，并且以这样的方式交织在一起，以至于心灵是身体的许多基本功能之一。他认为，人类的行为是由他们的欲望驱动的，而这些欲望是由他们的身体需求所引起的。他还提出了一些关于人类情感和情绪的理论，例如他认为愤怒是一种消极的情感，因为它会导致人们做出不理智的决定  。'

In [ ]:
chat_llm_chain.predict(human_input="哪里能买到这本书？")